Refactoring the following functionality using parallel processing:

```python
for dataset in datasets:
    mode_action = dataset.groupby('Component')['Action'].agg(lambda x: x.mode())

    interactions_per_user = dataset.groupby(['Component', 'User_ID']).size()\
        .reset_index(name='Interaction_count')
    mean_interactions_per_user = interactions_per_user.groupby('Component')['Interaction_count'].mean()

    median_date_of_component_interaction = dataset.groupby('Component')['Date'].median()

    results.append(pd.DataFrame({'Mode Action': mode_action,
                            'Mean Interactions per User': mean_interactions_per_user,
                            'Median Date of Interaction': median_date_of_component_interaction}).reset_index())
    
return results
```

In [1]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import time
from os import path

%run pickling_db_funcs.ipynb

In [4]:
def chunk_df(df: pd.DataFrame, chunksize):
    return [df.iloc[i:i+chunksize] for i in range(0, len(df), chunksize)]

def json_chunk(chunk:pd.DataFrame, index):
    chunk.to_json(path.join(DATA_FOLDER, 'json_conc', f'_{index}.json'), orient='records')

def thread_save_to_json(df):
    chunks = chunk_df(df, len(df) // 6)

    with ThreadPoolExecutor() as exe:
        futures = [exe.submit(json_chunk, chunk, i) for i, chunk in enumerate(chunks)]

def normal_save_to_json(df):
    df.to_json(path.join(DATA_FOLDER, 'json_conc', 'test.json'), orient='records')

def benchmark(dataset, func, reps=20):
    times = []
    for _ in range(reps):
        start_time = time.time()
        func(dataset)
        times.append(time.time() - start_time)
    
    avg_time = sum(times) / reps
    print(f'{func.__name__} - Average Time: {avg_time:.4f} seconds over {reps} runs')

In [5]:
test_data = pd.DataFrame(read_pickle_backup('merged'))
test_data = pd.concat([test_data] * 10)
benchmark(test_data, thread_save_to_json, reps=50)
benchmark(test_data, normal_save_to_json, reps=50)

thread_save_to_json - Average Time: 1.3939 seconds over 50 runs
normal_save_to_json - Average Time: 1.4492 seconds over 50 runs
